In [ ]:
import os
import json
from datetime import datetime

def print_json(data, indent=4, sort_keys=True):
    """
    美化输出 JSON 数据
    :param data: 待序列化的 Python 对象（字典/列表）
    :param output_file: 输出文件路径（None 则打印到控制台）
    :param indent: 缩进空格数
    :param sort_keys: 是否按键排序
    """
    # 自定义序列化处理
    def serialize(obj):
        if isinstance(obj, datetime):
            return obj.isoformat()  # ISO 格式时间
        elif isinstance(obj, (int, float, bool, str, list, dict, type(None))):
            return obj
        else:
            return str(obj)  # 其他类型转为字符串

    # 生成美化的 JSON 字符串
    json_str = json.dumps(
        data,
        indent=indent,
        sort_keys=sort_keys,
        ensure_ascii=False,
        default=serialize
    )

    print(json_str)

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

OPENAI_MODEL = os.getenv('OPENAI_MODEL')

llm = ChatOpenAI(
        model="OPENAI_MODEL", 
        temperature=0.1,
        max_tokens=50
    )

# 1.对话消息历史组件基础.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
@Time    : 2024/5/22 9:22
@Author  : thezehui@gmail.com
@File    : 1.对话消息历史组件基础.py
"""
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_history = InMemoryChatMessageHistory()

chat_history.add_user_message("你好，我是慕小课，你是谁？")
chat_history.add_ai_message("你好，我是ChatGPT，有什么可以帮到您的？")

print_json(chat_history.messages)

# 2.文件对话消息历史组件实现记忆.py

In [ ]:
from langchain.memory import FileChatMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate

# 1. 创建文件聊天历史
history = FileChatMessageHistory("chat-history.json")

# 2. 查看历史消息
print("历史消息：", history.messages)

# 3. 添加消息
history.add_user_message("你好，我是张三")
history.add_ai_message("你好张三！有什么可以帮助你的？")

# 4. 与LLM结合使用
template = ChatPromptTemplate.from_messages([
    ("system", "你是一个友好的助手。"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# 获取历史 + 新输入
messages = history.messages + [HumanMessage(content="今天天气怎么样？")]
response = llm.invoke(messages)
history.add_ai_message(response.content)

print("最新回复：", response.content)